In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('cleaned_data.csv')
# leaving only entries with salary
df = df[~df.salary.isna()]

In [3]:
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [4]:
df.head()

,vacancy_id,company,description,schedule,city,salary
1,6118569,"Мультиметод, ООО",международная школа восстановительной медицин...,полная занятость,Киев,20000.0
2,5148817,m1-shop.ru,компания m1-shop - рекламная сеть с одним из ...,удаленная работа,Киев,10300.0
5,7667113,Мозырь Украина,ведущий менеджер по оптовым продажам нефтепр...,полная занятость,Киев,30000.0
13,874023,Долфі-Україна,національна дистрибуційна компанія «долфі-укр...,полная занятость,Киев,12000.0
19,2618585,"Макси Маркетинг, ООО",робота в представництві міжнародної фінансово...,полная занятость,Киев,20000.0


In [5]:
small_cities = df.city.value_counts().index[df.city.value_counts() < 10]
df.city = df.city.apply(lambda x: 'Small city' if x in small_cities else x)

In [6]:
city_ohe = OneHotEncoder()
city_encoded = city_ohe.fit_transform(df.city.values.reshape(-1, 1)).todense()
city_df = pd.DataFrame(city_encoded, columns=['city.'+cat for cat in city_ohe.categories_], index=df.index)

In [7]:
schedule_ohe = OneHotEncoder()
schedule_encoded = schedule_ohe.fit_transform(df.schedule.values.reshape(-1, 1)).todense()
schedule_df = pd.DataFrame(schedule_encoded, columns=['schedule.'+cat for cat in schedule_ohe.categories_], index=df.index)

In [8]:
small_companies = df.company.value_counts().index[df.company.value_counts() < 8]
df.company = df.company.apply(lambda x: 'Noname' if x in small_companies else x)

In [9]:
company_ohe = OneHotEncoder()
company_encoded = company_ohe.fit_transform(df.company.values.reshape(-1, 1)).todense()
company_df = pd.DataFrame(company_encoded, columns=['company.'+cat for cat in company_ohe.categories_], index=df.index)

In [10]:
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer, TfidfVectorizer

In [11]:
tf_idf = TfidfVectorizer()
tf_idf_description = tf_idf.fit_transform(df.description)
description_df = pd.DataFrame(tf_idf_description.todense(), index=df.index)

In [12]:
feature_array = np.array(tf_idf.get_feature_names())

In [13]:
indices = np.argsort(tf_idf.idf_)[::-1]
features = np.array(tf_idf.get_feature_names())

In [14]:
n = 50
top_n = features[indices[:n]]

In [15]:
top_n

array(['взаємодієш', 'наполегливим', 'приведём', 'приведе', 'приватні',
       'выросло', 'приватними', 'приватний', 'высвобождают', 'высланы',
       'наполегливий', 'выслушает', 'привабливих', 'привели', 'высоки',
       'приваблива', 'наполегливих', 'наполегливо', 'прибыльность',
       'высококачественным', 'прибыльное', 'прибыльно',
       'высококачественных', 'высококвалифицированного', 'привезены',
       'приверженность', 'прибутку', 'наповненні', 'вырабатывать',
       'привлекательный', 'выработанной', 'выражается',
       'привлекательности', 'выражайся', 'привлекает', 'привлекаем',
       'привилегии', 'выраженные', 'выразительным', 'привертати',
       'вырастает', 'вырастают', 'вырастет', 'приветствия', 'вырастим',
       'вырастут', 'приветливом', 'напоит', 'выращивания',
       'привествуется'], dtype='<U75')

In [16]:
train_df = pd.concat([city_df, schedule_df, company_df], axis=1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(train_df, df.salary, test_size=0.25)

In [18]:
X_train.shape, X_test.shape

((1992, 80), (664, 80))

In [19]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(100)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)

In [20]:
from sklearn.metrics import r2_score, mean_absolute_error
print(f"r2_score: {r2_score(y_test, y_pred)}, MAE: {mean_absolute_error(y_test, y_pred)}")

r2_score: 0.21804289523158915, MAE: 4362.554254163258


In [37]:
X_train, X_test, y_train, y_test = train_test_split(description_df, df.salary, test_size=0.25)

In [38]:
from sklearn.linear_model import Ridge

In [41]:
ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

In [42]:
print(f"r2_score: {r2_score(y_test, y_pred)}, MAE: {mean_absolute_error(y_test, y_pred)}")

r2_score: 0.46467815839939397, MAE: 3759.765819584498


In [43]:
combined_df = pd.concat([train_df, description_df], axis=1)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(combined_df, df.salary, test_size=0.25)

In [54]:
ridge_all = Ridge(0.05)
ridge_all.fit(X_train, y_train)
y_pred = ridge_all.predict(X_test)

In [55]:
print(f"r2_score: {r2_score(y_test, y_pred)}, MAE: {mean_absolute_error(y_test, y_pred)}")

r2_score: 0.4721411814520189, MAE: 3333.5291729802625


In [56]:
for i in range(20):
    print(f"True salary: {y_test.values[i]} | Predicted: {y_pred[i]}")

True salary: 12000.0 | Predicted: 10354.724641123265
True salary: 10000.0 | Predicted: 21201.704409688315
True salary: 8000.0 | Predicted: 12594.049083234839
True salary: 10000.0 | Predicted: 11666.082748608265
True salary: 13000.0 | Predicted: 14919.80225536205
True salary: 6000.0 | Predicted: 8048.954485650387
True salary: 6000.0 | Predicted: 11212.153738043036
True salary: 8000.0 | Predicted: 7728.053034598002
True salary: 15000.0 | Predicted: 16201.48391144082
True salary: 7000.0 | Predicted: 6364.576997247539
True salary: 21759.0 | Predicted: 18759.17719023483
True salary: 35000.0 | Predicted: 17544.29280038343
True salary: 6000.0 | Predicted: 6046.8556258397475
True salary: 12000.0 | Predicted: 13365.553872738361
True salary: 6000.0 | Predicted: 6791.503222928202
True salary: 30000.0 | Predicted: 21132.326355401266
True salary: 8000.0 | Predicted: 12969.435820081713
True salary: 8000.0 | Predicted: 7911.745019599317
True salary: 5000.0 | Predicted: 13954.099423423702
True salary: